<a href="https://colab.research.google.com/github/zlw1225/MPLC_CUDA/blob/main/colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git --version

git version 2.34.1


In [ ]:
!git clone https://github.com/zlw1225/MPLC_CUDA.git

Cloning into 'MPLC_CUDA'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 9 (delta 0), reused 9 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (9/9), 17.57 MiB | 12.41 MiB/s, done.


In [19]:
import os
os.getcwd()
path="/content/MPLC_CUDA"
os.chdir(path)
os.getcwd()

'/content/MPLC_CUDA'

In [18]:
!python MPLC_CUDA2.py --iterations 2

[MPLC2] Using device: cpu
Traceback (most recent call last):
  File "/content/MPLC_CUDA/MPLC_CUDA2.py", line 251, in <module>
    phi_prop = propagate_HK(phi_prop, kz_torch_list[l], -d)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/functools.py", line 912, in wrapper
    return dispatch(args[0].__class__)(*args, **kw)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/MPLC_CUDA/utils.py", line 116, in _
    FieldOut = ifft2(FieldOut_FT)
               ^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/functools.py", line 912, in wrapper
    return dispatch(args[0].__class__)(*args, **kw)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/MPLC_CUDA/utils.py", line 78, in ifft2_torch
    return torch.fft.fftshift(torch.fft.ifft2(torch.fft.ifftshift(x, dim=(-1, -2)), norm="ortho"), dim=(-1, -2))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
KeyboardI